# Récupération des données pour l'étude départementale

## 1- Importation des librairies externes et classiques

In [1]:
# Importation des librairies externes
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data@master --ignore-install PyYAML
!pip install python-Levenshtein
!pip install openpyxl

  Cloning https://github.com/InseeFrLab/Py-Insee-Data (to revision master) to /tmp/pip-req-build-7smw83gc
  Running command git clone --filter=blob:none -q https://github.com/InseeFrLab/Py-Insee-Data /tmp/pip-req-build-7smw83gc
  Resolved https://github.com/InseeFrLab/Py-Insee-Data to commit 52139da402bc33da6b8892d7d3208f1d2ddfaa02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached pandas-1.3.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached pathlib2-2.3.6-py2.py3-none-any.whl (17 kB)
  Using cached

In [2]:
# Importation des librairies 
import requests
import pandas as pd
import pynsee
import pynsee.download
from pynsee.sirene import *
import time
from tqdm import tqdm
from pynsee.utils.init_conn import init_conn
import os
from zipfile import ZipFile, Path
import io
import numpy as np

In [3]:
# token API insee
init_conn(insee_key=os.environ['INSEE_KEY'], insee_secret=os.environ['INSEE_SECRET'])

## 2- Téléchargement des données de SIRENE

In [4]:
### Téléchargement des données Sirene sur les établissements Siret de boucherie-charcuterie (code 4722Z) à l'instant T
# Téléchargement via Insee sirene data  by sirene.fr website

session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)


In [5]:
donnees_sirene_fr.shape
donnees_sirene_fr.sample(10)

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,...,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
14774,822557690,00010,82255769000010,O,2016-10-03,03,2019,4722ZB,2021-10-27T09:28:21,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
5696,448980938,00022,44898093800022,O,2012-05-01,NaN,NaN,NaN,2021-02-23T13:30:57,false,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
3098,401859517,00021,40185951700021,O,2016-09-30,NN,NaN,4722ZA,2021-02-23T11:25:57,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
19382,894225960,00018,89422596000018,O,2021-02-15,NaN,NaN,4722ZB,2021-05-04T20:31:20,true,...,NaN,NaN,A,BOUCHERIE DES HALLES DIVAISE,NaN,NaN,BOUCHERIE DES HALLES DIVAISE,47.22Z,NAFRev2,O
11158,752568279,00026,75256827900026,O,2012-07-01,00,2019,4722ZB,2021-10-27T08:52:12,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
19647,899061238,00015,89906123800015,O,2021-04-29,NaN,NaN,4722ZA,2021-05-26T08:47:06,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
17035,844777763,00013,84477776300013,O,2019-01-01,NaN,NaN,4722ZA,2020-07-03T08:37:42,true,...,NaN,NaN,A,NaN,NaN,NaN,BOUCHERIE DU FAUBOURG,47.22Z,NAFRev2,O
18692,885239871,00019,88523987100019,O,2020-07-17,NaN,NaN,4722ZB,2020-10-19T15:34:59,true,...,NaN,NaN,A,CHEZ ROMAIN,NaN,NaN,NaN,47.22Z,NAFRev2,O
1860,380732362,00054,38073236200054,O,2015-10-01,03,2019,NaN,2021-10-27T08:08:44,false,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
18772,887993640,00010,88799364000010,O,2020-07-18,NaN,NaN,NaN,2021-04-13T16:08:23,true,...,NaN,NaN,A,BOUCHERIE DE L'OLIVIER / SH MARKET,NaN,NaN,BOUCHERIE DE L'OLIVIER / SH MARKET,47.22Z,NAFRev2,O


* les données de Sirene.fr sont des données d'entreprises France entière. Pour l'étude, le nombre d'établissement par département est considéré :

In [6]:
pd.set_option('display.max_columns', None)
donnees_sirene_fr.sample(10)

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,etatAdministratifUniteLegale,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,categorieJuridiqueUniteLegale,denominationUniteLegale,sigleUniteLegale,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,sexeUniteLegale,nomUniteLegale,nomUsageUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,prenomUsuelUniteLegale,pseudonymeUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,identifiantAssociationUniteLegale,economieSocialeSolidaireUniteLegale,caractereEmployeurUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nicSiegeUniteLegale,dateDernierTraitementUniteLegale,categorieEntreprise,anneeCategorieEntreprise,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
3894,417602331,00023,41760233100023,O,2000-01-04,NaN,NaN,4722ZB,2019-11-14T14:00:53,false,A,O,NaN,1997-10-01,1000,NaN,NaN,BOUCHERIE FRANCOIS,NaN,NaN,M,FRANCOIS,NaN,SEBASTIEN,NaN,NaN,NaN,SEBASTIEN,NaN,47.22Z,NAFRev2,NaN,NaN,O,01,2019,00015,2021-10-27T08:39:06,PME,2019,CENTRE COMMERCIAL BELLEVUE,8,NaN,RUE,DU DOC GALLAVARDIN,69800,SAINT-PRIEST,NaN,NaN,69290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
10696,539696401,00014,53969640100014,O,2012-01-20,NaN,NaN,4722ZB,2019-12-17T04:02:14,true,A,O,NaN,2012-01-20,5499,LA MAISON DU BOUCHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,NaN,NaN,N,NaN,NaN,00014,2020-07-09T10:28:14,PME,2019,NaN,102,NaN,RTE,D ALBI,31200,TOULOUSE,NaN,NaN,31555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,LA MAISON DU BOUCHER,47.22Z,NAFRev2,N
7714,498675750,00067,49867575000067,O,2012-03-01,NN,NaN,NaN,2021-03-18T14:37:35,false,A,O,NaN,2007-06-01,5499,FLORIACK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,NaN,N,O,11,2019,00018,2021-10-27T08:37:08,PME,2019,NaN,618,NaN,RTE,NATIONALE,65120,BAREGES,NaN,NaN,65481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
1304,343188819,00043,34318881900043,O,2015-12-28,NaN,NaN,NaN,2019-08-14T03:35:15,true,A,O,NaN,1987-12-14,5710,SBK SARL,NaN,L'ORIENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.10A,NAFRev2,NaN,N,N,02,2019,00043,2021-10-27T08:35:48,PME,2019,NaN,70,NaN,RUE,DES SAUVETEURS,76600,LE HAVRE,NaN,NaN,76351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
3736,414544197,00026,41454419700026,O,2014-03-31,NaN,NaN,NaN,2021-04-29T03:57:11,true,A,O,NaN,1997-10-25,1000,NaN,NaN,NaN,NaN,NaN,M,LEBRANCHU,NaN,THIERRY,ROBERT,NaN,NaN,THIERRY,NaN,47.22Z,NAFRev2,NaN,NaN,O,NN,NaN,00026,2021-04-29T03:57:11,PME,2019,NaN,9,NaN,PL,DE L

In [7]:
donnees_sirene_fr["departement"]=donnees_sirene_fr.codeCommuneEtablissement.str[:2]

donnees_sirene_fr["departement"]=np.where(donnees_sirene_fr.codeCommuneEtablissement.str[:2]=='97', donnees_sirene_fr.codeCommuneEtablissement.str[:3],donnees_sirene_fr.codeCommuneEtablissement.str[:2])
donnees_sirene_fr_agregees_par_departement = donnees_sirene_fr.groupby("departement").size().reset_index(name='NbBoucheriesParDep').set_index("departement")
donnees_sirene_fr_agregees_par_departement.describe()

,NbBoucheriesParDep
count,104.000000
mean,193.730769
std,168.350145
min,3.000000
25%,95.750000
50%,140.000000
75%,228.500000
max,877.000000


## 3- Téléchargement des données du ministère de l'intérieur

In [8]:
#Téléchargement des données de crimes et délits rapportés déclarés auprès des commissariats 
url_datagouv_min_interieur="https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520"

donnees_police_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services PN 2018', skiprows=[1,2])
donnees_gendarmerie_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services GN 2018', skiprows=[1,2])

del donnees_police_nationale_2018[donnees_police_nationale_2018.columns[0]]
del donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[0]]

donnees_police_nationale_2018.set_index('Départements')
donnees_gendarmerie_nationale_2018.set_index('Départements')
#donnees_police_nationale_2018.head(10)
#donnees_gendarmerie_nationale_2018.head(10)



,01,01.1,01.2,01.3,02,02.1,02.2,02.3,02.4,03,03.1,03.2,04,04.1,04.2,04.3,05,05.1,06,06.1,06.2,06.3,06.4,07,07.1,07.2,08,08.1,08.2,08.3,08.4,09,09.1,09.2,10,10.1,10.2,11,11.1,11.2,12,12.1,12.2,13,13.1,13.2,13.3,13.4,14,14.1,14.2,14.3,14.4,14.5,15,15.1,15.2,16,16.1,16.2,17,17.1,17.2,17.3,17.4,18,18.1,18.2,19,19.1,21,21.1,21.2,21.3,22,22.1,22.2,22.3,23,23.1,24,24.1,24.2,24.3,25,25.1,25.2,26,26.1,26.2,26.3,27,27.1,27.2,27.3,27.4,28,28.1,28.2,28.3,29,29.1,29.2,29.3,29.4,29.5,2A,2A.1,2A.2,2B,2B.1,2B.2,2B.3,30,30.1,30.2,30.3,30.4,31,31.1,31.2,31.3,31.4,32,32.1,33,33.1,33.2,33.3,33.4,33.5,33.6,34,34.1,34.2,34.3,34.4,35,35.1,35.2,35.3,35.4,36,36.1,36.2,37,37.1,37.2,37.3,38,38.1,38.2,38.3,38.4,38.5,38.6,39,39.1,39.2,40,40.1,40.2,41,41.1,41.2,42,42.1,42.2,43,43.1,43.2,44,44.1,44.2,44.3,44.4,44.5,45,45.1,45.2,45.3,46,46.1,46.2,47,47.1,47.2,47.3,48,48.1,49,49.1,49.2,49.3,50,50.1,50.2,50.3,51,51.1,51.2,51.3,52,52.1,52.2,53,53.1,54,54.1,54.2,54.3,55,55.1,56,56.1,56.2,56.3,57,57.1,57.2,57.3,57.4,57.5,58,58.1,58.2,59,59.1,59.2,59.3,59.4,59.5,59.6,60,60.1,60.2,60.3,60.4,60.5,61,61.1,61.2,62,62.1,62.2,62.3,62.4,62.5,63,63.1,63.2,63.3,63.4,63.5,64,64.1,64.2,64.3,64.4,65,65.1,65.2,66,66.1,66.2,66.3,67,67.1,67.2,67.3,67.4,67.5,68,68.1,68.2,68.3,69,69.1,69.2,69.3,69.4,70,70.1,71,71.1,71.2,71.3,71.4,72,72.1,72.2,73,73.1,73.2,74,74.1,74.2,74.3,74.4,76,76.1,76.2,76.3,76.4,76.5,77,77.1,77.2,77.3,77.4,78,78.1,78.2,79,79.1,79.2,80,80.1,80.2,80.3,81,81.1,81.2,82,82.1,83,83.1,83.2,83.3,83.4,84,84.1,84.2,84.3,85,85.1,85.2,86,86.1,86.2,87,87.1,87.2,88,88.1,88.2,89,89.1,89.2,91,91.1,91.2,95,95.1,95.2,971,971.1,971.2,972,972.1,972.2,973,973.1,973.2,974,974.1,974.2,978,987,987.1,988,988.1,988.2,988.3
Départements,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Homicides pour voler et à l'occasion de vols,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Homicides pour d'autres motifs,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0

In [9]:
donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[1:]]=donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[1:]].fillna(0).astype(int)
donnees_gendarmerie_nationale_2018

,Départements,01,01.1,01.2,01.3,02,02.1,02.2,02.3,02.4,03,03.1,03.2,04,04.1,04.2,04.3,05,05.1,06,06.1,06.2,06.3,06.4,07,07.1,07.2,08,08.1,08.2,08.3,08.4,09,09.1,09.2,10,10.1,10.2,11,11.1,11.2,12,12.1,12.2,13,13.1,13.2,13.3,13.4,14,14.1,14.2,14.3,14.4,14.5,15,15.1,15.2,16,16.1,16.2,17,17.1,17.2,17.3,17.4,18,18.1,18.2,19,19.1,21,21.1,21.2,21.3,22,22.1,22.2,22.3,23,23.1,24,24.1,24.2,24.3,25,25.1,25.2,26,26.1,26.2,26.3,27,27.1,27.2,27.3,27.4,28,28.1,28.2,28.3,29,29.1,29.2,29.3,29.4,29.5,2A,2A.1,2A.2,2B,2B.1,2B.2,2B.3,30,30.1,30.2,30.3,30.4,31,31.1,31.2,31.3,31.4,32,32.1,33,33.1,33.2,33.3,33.4,33.5,33.6,34,34.1,34.2,34.3,34.4,35,35.1,35.2,35.3,35.4,36,36.1,36.2,37,37.1,37.2,37.3,38,38.1,38.2,38.3,38.4,38.5,38.6,39,39.1,39.2,40,40.1,40.2,41,41.1,41.2,42,42.1,42.2,43,43.1,43.2,44,44.1,44.2,44.3,44.4,44.5,45,45.1,45.2,45.3,46,46.1,46.2,47,47.1,47.2,47.3,48,48.1,49,49.1,49.2,49.3,50,50.1,50.2,50.3,51,51.1,51.2,51.3,52,52.1,52.2,53,53.1,54,54.1,54.2,54.3,55,55.1,56,56.1,56.2,56.3,57,57.1,57.2,57.3,57.4,57.5,58,58.1,58.2,59,59.1,59.2,59.3,59.4,59.5,59.6,60,60.1,60.2,60.3,60.4,60.5,61,61.1,61.2,62,62.1,62.2,62.3,62.4,62.5,63,63.1,63.2,63.3,63.4,63.5,64,64.1,64.2,64.3,64.4,65,65.1,65.2,66,66.1,66.2,66.3,67,67.1,67.2,67.3,67.4,67.5,68,68.1,68.2,68.3,69,69.1,69.2,69.3,69.4,70,70.1,71,71.1,71.2,71.3,71.4,72,72.1,72.2,73,73.1,73.2,74,74.1,74.2,74.3,74.4,76,76.1,76.2,76.3,76.4,76.5,77,77.1,77.2,77.3,77.4,78,78.1,78.2,79,79.1,79.2,80,80.1,80.2,80.3,81,81.1,81.2,82,82.1,83,83.1,83.2,83.3,83.4,84,84.1,84.2,84.3,85,85.1,85.2,86,86.1,86.2,87,87.1,87.2,88,88.1,88.2,89,89.1,89.2,91,91.1,91.2,95,95.1,95.2,971,971.1,971.2,972,972.1,972.2,973,973.1,973.2,974,974.1,974.2,978,987,987.1,988,988.1,988.2,988.3
0,Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0
1,Homicides pour d'autres motifs,0,0,0,0,0,2,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,2,1,0,0,1,2,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,2,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,1,2,0,1,1,0,1,1,0,0,0,1,0,2,0,3,0,0,0,0,1,1,1,2,0,1,0,2,0,1,1,2,1,0,2,1,0,1,0,0,0,1,2,1,0,0,0,0,0,2,1,2,0,0,1,0,0,1,2,3,1,2,3,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,1,1,0,0,0,0,0,1,2,2,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,2,1,1,0,0,0,0,0,0,1,1,1,2,0,1,0,0,3,0,0,1,0,0,0,1,0,0,1,0,0,2,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,1,3,1,0,3,1,0,0,0,1,0,0,0,1,1,3,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,3,2,0,1,0,0,0,0,1,2,3,3,1,0,2,0,0,2,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,1,1,1,1,0,0,1,2,2,0,2,5,3,5,5,1,2,3,0,2,1,0,5,3,2
2,Tentatives d'homicides pour voler et à l'occas...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0

In [10]:
donnees_police_nationale_2018_par_dep=donnees_police_nationale_2018.T
donnees_police_nationale_2018_par_dep.columns = donnees_police_nationale_2018_par_dep.iloc[0]
donnees_police_nationale_2018_par_dep=donnees_police_nationale_2018_par_dep.iloc[1: , :]

donnees_police_nationale_2018_par_dep.drop(donnees_police_nationale_2018_par_dep[donnees_police_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
donnees_police_nationale_2018_par_dep["departement"]=np.where(donnees_police_nationale_2018_par_dep.index.str[:2]=='97', donnees_police_nationale_2018_par_dep.index.str[:3],donnees_police_nationale_2018_par_dep.index.str[:2])

donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.groupby("departement").sum() 
donnees_police_nationale_2018_par_dep

Départements,Règlements de compte entre malfaireurs,Homicides pour voler et à l'occasion de vols,Homicides pour d'autres motifs,Tentatives d'homicides pour voler et à l'occasion de vols,Tentatives homicides pour d'autres motifs,Coups et blessures volontaires suivis de mort,Autres coups et blessures volontaires criminels ou correctionnels,Prises d'otages à l'occasion de vols,Prises d'otages dans un autre but,Sequestrations,Menaces ou chantages pour extorsion de fonds,Menaces ou chantages dans un autre but,Atteintes à la dignité et à la personnalité,Violations de domicile,Vols à main armée contre des établissements financiers,Vols à main armée contre des éts industriels ou commerciaux,Vols à main armée contre des entreprises de transports de fonds,Vols à main armée contre des particuliers à leur domicile,Autres vols à main armée,"Vols avec armes blanches contre des établissements financiers,commerciaux ou industriels",Vols avec armes blanches contre des particuliers à leur domicile,Autres vols avec armes blanches,"Vols violents sans arme contre des établissements financiers,commerciaux ou industriels",Vols violents sans arme contre des particuliers à leur domicile,Vols violents sans arme contre des femmes sur voie publique ou autre lieu public,Vols violents sans arme contre d'autres victimes,Cambriolages de locaux d'habitations principales,Cambriolages de résidences secondaires,"Camb.de locaux industriels, commerciaux ou financiers",Cambriolages d'autres lieux,Vols avec entrée par ruse en tous lieux,Vols à la tire,Vols à l'étalage,Vols de véhicules de transport avec frêt,Vols d'automobiles,Vols de véhicules motorisés à 2 roues,Vols à la roulotte,Vols d''accessoires sur véhicules à moteur immatriculés,Vols simples sur chantier,Vols simples sur exploitations agricoles,Autres vols simples contre des établissements publics ou privés,Autres vols simples contre des particuliers dans deslocaux privés,Autres vols simples contre des particuliers dans des locaux ou lieux publics,Recels,Proxénétisme,Viols sur des majeur(e)s,Viols sur des mineur(e)s,Harcèlements sexuels et autres agressions sexuelles contre des majeur(e)s,Harcèlements sexuels et autres agressions sexuelles contre des mineur(e)s,Atteintes sexuelles,Homicides commis contre enfants de moins de 15 ans,"Violences, mauvais traitements et abandons d'enfants.",Délits au sujet de la garde des mineurs,Non versement de pension alimentaire,Trafic et revente sans usage de stupéfiants,Usage-revente de stupéfiants,Usage de stupéfiants,Autres infractions à la législation sur les stupéfiants,Délits de débits de boissons et infraction à la règlementation sur l'alcool et le tabac,Fraudes alimentaires et infractions à l'hygiène,Autres délits contre santé publique et la réglementation des professions médicales,Incendies volontaires de biens publics,Incendies volontaires de biens privés,Attentats à l'explosif contre des biens publics,Attentats à l'explosif contre des biens privés,Autres destructions er dégradations de biens publics,Autres destructions er dégradations de biens privés,Destructions et dégradations de véhicules privés,Infractions aux conditions générales d'entrée et de séjour des étrangers,"Aide à l'entrée, à la circulation et au séjour des étrangers",Autres infractions à la police des étrangers,Outrages à dépositaires autorité,Violences à dépositaires autorité,Port ou détention armes prohibées,Atteintes aux intérêts fondamentaux de la Nation,Délits des courses et des jeux,Délits interdiction de séjour et de paraître,"Destructions, cruautés et autres délits envers les animaux",Atteintes à l'environnement,Chasse et pêche,Faux documents d'identité,Faux documents concernant la circulation des véhicules,Autres faux documents administratifs,Faux en écriture publique et authentique,Autres faux en écriture,Fausse monnaie,Contrefaçons et fraudes industrielles et commerciales,Contrefaçons littéraires et artistique,Falsification et usages de chèques volés,Falsification et usages de cartes de 

* Agrégation par département, transposition et fusion :

In [11]:
donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_police_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Police Nationale"},inplace=True)

In [12]:
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018.T
donnees_gendarmerie_nationale_2018_par_dep.columns = donnees_gendarmerie_nationale_2018_par_dep.iloc[0]
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018_par_dep.iloc[1: , :]

donnees_gendarmerie_nationale_2018_par_dep.drop(donnees_gendarmerie_nationale_2018_par_dep[donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
donnees_gendarmerie_nationale_2018_par_dep["departement"]=np.where(donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='97', donnees_gendarmerie_nationale_2018_par_dep.index.str[:3],donnees_gendarmerie_nationale_2018_par_dep.index.str[:2])

donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.groupby("departement").sum() 
donnees_gendarmerie_nationale_2018_par_dep

Départements,Homicides pour voler et à l'occasion de vols,Homicides pour d'autres motifs,Tentatives d'homicides pour voler et à l'occasion de vols,Tentatives homicides pour d'autres motifs,Coups et blessures volontaires suivis de mort,Autres coups et blessures volontaires criminels ou correctionnels,Prises d'otages à l'occasion de vols,Prises d'otages dans un autre but,Sequestrations,Menaces ou chantages pour extorsion de fonds,Menaces ou chantages dans un autre but,Atteintes à la dignité et à la personnalité,Violations de domicile,Vols à main armée contre des établissements financiers,Vols à main armée contre des éts industriels ou commerciaux,Vols à main armée contre des entreprises de transports de fonds,Vols à main armée contre des particuliers à leur domicile,Autres vols à main armée,"Vols avec armes blanches contre des établissements financiers,commerciaux ou industriels",Vols avec armes blanches contre des particuliers à leur domicile,Autres vols avec armes blanches,"Vols violents sans arme contre des établissements financiers,commerciaux ou industriels",Vols violents sans arme contre des particuliers à leur domicile,Vols violents sans arme contre des femmes sur voie publique ou autre lieu public,Vols violents sans arme contre d'autres victimes,Cambriolages de locaux d'habitations principales,Cambriolages de résidences secondaires,"Camb.de locaux industriels, commerciaux ou financiers",Cambriolages d'autres lieux,Vols avec entrée par ruse en tous lieux,Vols à la tire,Vols à l'étalage,Vols de véhicules de transport avec frêt,Vols d'automobiles,Vols de véhicules motorisés à 2 roues,Vols à la roulotte,Vols d''accessoires sur véhicules à moteur immatriculés,Vols simples sur chantier,Vols simples sur exploitations agricoles,Autres vols simples contre des établissements publics ou privés,Autres vols simples contre des particuliers dans deslocaux privés,Autres vols simples contre des particuliers dans des locaux ou lieux publics,Recels,Proxénétisme,Viols sur des majeur(e)s,Viols sur des mineur(e)s,Harcèlements sexuels et autres agressions sexuelles contre des majeur(e)s,Harcèlements sexuels et autres agressions sexuelles contre des mineur(e)s,Atteintes sexuelles,Homicides commis contre enfants de moins de 15 ans,"Violences, mauvais traitements et abandons d'enfants.",Délits au sujet de la garde des mineurs,Non versement de pension alimentaire,Trafic et revente sans usage de stupéfiants,Usage-revente de stupéfiants,Usage de stupéfiants,Autres infractions à la législation sur les stupéfiants,Délits de débits de boissons et infraction à la règlementation sur l'alcool et le tabac,Fraudes alimentaires et infractions à l'hygiène,Autres délits contre santé publique et la réglementation des professions médicales,Incendies volontaires de biens publics,Incendies volontaires de biens privés,Attentats à l'explosif contre des biens publics,Attentats à l'explosif contre des biens privés,Autres destructions er dégradations de biens publics,Autres destructions er dégradations de biens privés,Destructions et dégradations de véhicules privés,Infractions aux conditions générales d'entrée et de séjour des étrangers,"Aide à l'entrée, à la circulation et au séjour des étrangers",Autres infractions à la police des étrangers,Outrages à dépositaires autorité,Violences à dépositaires autorité,Port ou détention armes prohibées,Atteintes aux intérêts fondamentaux de la Nation,Délits des courses et des jeux,Délits interdiction de séjour et de paraître,"Destructions, cruautés et autres délits envers les animaux",Atteintes à l'environnement,Chasse et pêche,Faux documents d'identité,Faux documents concernant la circulation des véhicules,Autres faux documents administratifs,Faux en écriture publique et authentique,Autres faux en écriture,Fausse monnaie,Contrefaçons et fraudes industrielles et commerciales,Contrefaçons littéraires et artistique,Falsification et usages de chèques volés,Falsification et usages de cartes de crédit,Escroqueries et abus de confianc

In [13]:
donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_gendarmerie_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Gendarmerie Nationale"},inplace=True)

In [14]:
merged_crimes_delits_2018_par_dep = donnees_police_nationale_2018_par_dep.merge(donnees_gendarmerie_nationale_2018_par_dep, on="departement")
merged_crimes_delits_2018_par_dep

,Total des crimes et délits relevés par la Police Nationale,Total des crimes et délits relevés par la Gendarmerie Nationale
departement,,
01,5877,20210
02,12147,13695
03,7953,5463
04,2759,5608
05,2919,3672
...,...,...
95,66605,14018
971,9962,13493
972,8385,9447


## 4 - Téléchargement des revenus localisés

In [15]:
#Téléchargement des données Filosofi au niveau IRIS (millénisme 2018)
URL_FILO_DECL_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DEC_IRIS_2018.xlsx"
URL_FILO_DISP_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DISP_IRIS_2018.xlsx"

donnees_filo_decl_2018=pd.read_excel(URL_FILO_DECL_2018, sheet_name='IRIS_DEC', skiprows=5)
donnees_filo_disp_2018=pd.read_excel(URL_FILO_DISP_2018, sheet_name='IRIS_DISP', skiprows=5)

In [16]:
URL_FILO_ZIP_2018="https://www.insee.fr/fr/statistiques/fichier/5009236/base-cc-filosofi-2018_CSV_geo2021.zip"
#Télécharger le zip de l'URL
r=requests.get(URL_FILO_ZIP_2018)

open("base-cc-filosofi-2018_CSV_geo2021.zip", 'wb').write(r.content)
    
    
with ZipFile("base-cc-filosofi-2018_CSV_geo2021.zip",'r') as myzip:
    data = myzip.open("cc_filosofi_2018_COM-geo2021.CSV")
    data2 = myzip.open("cc_filosofi_2018_DEP-geo2021.CSV")
    
donnees_filo_communes_2018=pd.read_csv(data,sep=';')    
donnees_filo_departement_2018=pd.read_csv(data2,sep=';')    

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
donnees_filo_communes_2018.head(10)
donnees_filo_departement_2018.head(10)

,CODGEO,NBMENFISC18,NBPERSMENFISC18,MED18,PIMP18,TP6018,TP60AGE118,TP60AGE218,TP60AGE318,TP60AGE418,TP60AGE518,TP60AGE618,TP60TOL118,TP60TOL218,PACT18,PTSA18,PCHO18,PBEN18,PPEN18,PPAT18,PPSOC18,PPFAM18,PPMINI18,PPLOGT18,PIMPOT18,D118,D918,RD18
0,01,258018,618263,23420,53.6,10.3,14.9,11.8,11.4,10.1,7.5,7.2,5.0,22.0,78.8,72.0,2.8,4.0,24.9,9.6,4.0,1.9,1.2,0.9,-17.3,12900,44080,3.4
1,02,221370,512838,19690,43.6,18.4,29.7,22.8,21.2,18.4,11.6,9.5,8.4,36.2,68.4,59.3,3.1,6.0,30.6,8.3,7.9,2.9,3.2,1.8,-15.2,10730,33170,3.1
2,03,157528,325813,20360,44.0,15.5,25.3,19.7,18.2,16.1,10.6,10.2,8.3,31.5,60.8,52.5,2.6,5.7,37.6,10.5,6.6,2.0,3.0,1.6,-15.5,11360,33600,3.0
3,04,73620,154535,20580,45.6,16.8,24.9,20.9,20.3,17.2,12.3,11.3,9.1,29.4,60.6,50.5,3.4,6.7,35.7,14.6,5.7,1.9,2.4,1.4,-16.6,11060,35380,3.2
4,05,63474,133391,20880,47.5,13.9,19.9,16.9,16.6,14.6,9.6,10.2,7.9,24.7,64.0,53.0,3.3,7.7,35.5,11.4,5.3,1.9,2.1,1.3,-16.2,11800,35410,3.0
5,06,520415,1099477,22140,55.2,15.8,21.7,17.6,17.6,16.0,13.2,11.9,8.1,25.6,72.6,62.8,3.2,6.6,29.7,13.5,4.8,1.5,2.1,1.2,-20.6,11090,41770,3.8
6,07,141763,311917,20780,45.0,14.4,21.5,18.3,17.5,14.1,9.9,9.9,8.2,29.6,64.2,55.3,3.3,5.6,34.0,12.2,5.5,2.1,2.1,1.3,-15.9,11630,34440,3.0
7,08,117199,261731,19650,41.5,18.9,29.7,22.6,22.1,18.2,13.1,11.7,9.0,36.4,66.5,57.8,2.9,5.8,31.3,9.2,7.8,2.6,3.3,1.9,-14.8,10660,32960,3.1
8,09,68614,143949,19820,40.7,18.0,26.7,22.8,21.0,18.7,13.0,13.2,11.2,33.2,60.8,51.6,3.3,5.9,37.4,9.6,7.0,2.0,3.3,1.7,-14.8,10690,33090,3.1
9,10,130958,288559,20430,46.6,16.2,27.6,21.1,18.6,15.1,10.7,7.7,6.2,33.2,68.8,57.3,3.0,8.5,30.8,10.1,6.8,2.4,2.7,1.7,-16.5,11240,35650,3.2


In [18]:
donnees_filo_departement_2018=donnees_filo_departement_2018.set_index("CODGEO")

## 5 - Téléchargement des populations légales

In [19]:
#Téléchargement des populations légales du millénisme 2018 
URL_POP_LEG="https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_POP_LEG)

open("ensemble.zip", 'wb').write(r.content)
    
    
with ZipFile("ensemble.zip",'r') as myzip:
    data = myzip.open("Departements.csv")

donnees_pop_leg_18=pd.read_csv(data,sep=';')     

In [20]:
donnees_pop_leg_18.sample(10)

,CODREG,REG,CODDEP,DEP,NBARR,NBCAN,NBCOM,PMUN,PTOT
70,27,Bourgogne-Franche-Comté,70,Haute-Saône,2,17.0,539,236018,242280
39,27,Bourgogne-Franche-Comté,39,Jura,3,17.0,494,259746,268874
62,32,Hauts-de-France,62,Pas-de-Calais,7,39.0,890,1466743,1488821
53,52,Pays de la Loire,53,Mayenne,3,17.0,242,307084,316608
59,32,Hauts-de-France,59,Nord,6,41.0,648,2606234,2636884
2,84,Auvergne-Rhône-Alpes,03,Allier,3,19.0,317,337171,346156
43,84,Auvergne-Rhône-Alpes,43,Haute-Loire,3,19.0,257,227552,234434
98,3,Guyane,973,Guyane,2,NaN,22,276128,278472
58,27,Bourgogne-Franche-Comté,58,Nièvre,4,17.0,309,205828,211256
49,52,Pays de la Loire,49,Maine-et-Loire,4,21.0,177,815883,835856


In [21]:
donnees_pop_leg_18=donnees_pop_leg_18.set_index("CODDEP")

## 6 - Téléchargement des données d'emplois localisées

In [22]:
#Téléchargement des données d'emploi localisées
donnees_emploi_statut_18 = pynsee.download.telechargerDonnees("ESTEL_T201", date = "2018")
donnees_emploi_statut_18.sample(10)

donnees_emploi_sexe_18 = pynsee.download.telechargerDonnees("ESTEL_T202", date = "2018")
donnees_emploi_sexe_18.sample(10)


URL_TAUX_CHOMAGE_PAR_DEP="https://www.insee.fr/fr/statistiques/fichier/2012804/sl_etc_2021T2.xls"


donnees_chomage_par_departement_series_longues=pd.read_excel(URL_TAUX_CHOMAGE_PAR_DEP, sheet_name='Département', skiprows=3)


Data will be stored in the following location: /tmp/tmpe5133kmu


Downloading: 100%|██████████| 2.26M/2.26M [00:00<00:00, 5.83MiB/s]


No destination directory defined. Data have been written there: /tmp/tmppfwqbiz9


Extracting: 100%|██████████| 13.6M/13.6M [00:00<00:00, 198MB/s]


Data will be stored in the following location: /tmp/tmpx4ljvew_


Downloading: 100%|██████████| 2.26M/2.26M [00:01<00:00, 1.68MiB/s]


No destination directory defined. Data have been written there: /tmp/tmp4qg06kic


Extracting: 100%|██████████| 13.6M/13.6M [00:00<00:00, 182MB/s]
/opt/conda/lib/python3.9/site-packages/pynsee/download/__init__.py:73: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  return chargerDonnees(


In [23]:
donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues[donnees_chomage_par_departement_series_longues["Code"].str.isnumeric().fillna(False)]
donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.set_index("Code")

## 7 - Téléchargement du Code Officiel Géographique

* Pour des raisons de simplicité, nous travaillons en géographie 2021 :

In [24]:
#cog_commune = pynsee.download.telechargerDonnees("COG_COMMUNE", date = "dernier")
URL_COG_2021="https://www.insee.fr/fr/statistiques/fichier/5057840/commune2021-csv.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_COG_2021)

open("commune2021-csv.zip", 'wb').write(r.content)
    
    
with ZipFile("commune2021-csv.zip",'r') as myzip:
    data = myzip.open("commune2021.csv")

donnees_cog_2021=pd.read_csv(data)     

In [25]:
donnees_cog_2021

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,COM,01001,84.0,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,COM,01002,84.0,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,COM,01004,84.0,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,COM,01005,84.0,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,COM,01006,84.0,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
37737,COM,97613,6.0,976,976D,NaN,0,M TSANGAMOUJI,M'Tsangamouji,M'Tsangamouji,97613,NaN
37738,COM,97614,6.0,976,976D,NaN,1,OUANGANI,Ouangani,Ouangani,97610,NaN
37739,COM,97615,6.0,976,976D,NaN,0,PAMANDZI,Pamandzi,Pamandzi,97611,NaN
37740,COM,97616,6.0,976,976D,NaN,0,SADA,Sada,Sada,97612,NaN


## 8 - Fusion des données

In [34]:
donnees_2018=merged_crimes_delits_2018_par_dep.merge(donnees_filo_departement_2018,left_index=True,right_index=True)
donnees_2018.dtypes
#.merge(donnees_pop_leg_18)
#.merge(donnees_chomage_par_departement_series_longues).merge(donnees_sirene_fr_agregees_par_departement)


Total des crimes et délits relevés par la Police Nationale           int64
Total des crimes et délits relevés par la Gendarmerie Nationale      int64
NBMENFISC18                                                          int64
NBPERSMENFISC18                                                      int64
MED18                                                                int64
PIMP18                                                             float64
TP6018                                                             float64
TP60AGE118                                                         float64
TP60AGE218                                                         float64
TP60AGE318                                                         float64
TP60AGE418                                                         float64
TP60AGE518                                                         float64
TP60AGE618                                                         float64
TP60TOL118               

In [ ]:
donnees_filo_departement_2018